## PyTorch实现L1，L2正则化，以及Dropout
Ref:
1. https://iamtrask.github.io/2015/07/28/dropout/
2. https://morvanzhou.github.io/tutorials/machine-learning/torch/5-03-dropout/

### numpy实现Dropout

In [3]:
import numpy as np

X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([[0, 1, 1, 0]]).T

alpha, hidden_dim, dropout_percent, do_dropout = (0.5, 4, 0.2, True)
synapse_0 = 2 * np.random.random((3, hidden_dim)) - 1
synapse_1 = 2 * np.random.random((hidden_dim, 1)) - 1

for j in range(60000):
    layer_1 = (1 / (1 + np.exp(-(np.dot(X, synapse_0)))))
    if (do_dropout):
        layer_1 *= np.random.binomial([np.ones(
            (len(X), hidden_dim))], 1 - dropout_percent)[0] * (
                1.0 / (1 - dropout_percent))
    layer_2 = 1 / (1 + np.exp(-(np.dot(layer_1, synapse_1))))
    layer_2_delta = (layer_2 - y) * (layer_2 * (1 - layer_2))
    layer_1_delta = layer_2_delta.dot(synapse_1.T) * (layer_1 * (1 - layer_1))
    synapse_1 -= (alpha * layer_1.T.dot(layer_2_delta))
    synapse_0 -= (alpha * X.T.dot(layer_1_delta))

### PyTorch实现Dropout

In [13]:
from torch import nn

class SimpleNet(nn.Module):
    """
    定义了一个简单的三层全连接神经网络，每一层都是线性的
    """
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

class ActivationNet(nn.Module):
    """
    在上面的SimpleNet的基础上，在每层的输出部分添加了激活函数
    """
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(ActivationNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
        """
        这里的Sequential()函数的功能是将网络的层组合到一起。
        """

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
class DropoutNet(nn.Module):
    """
    在上面的ActivationNet的基础上，在每层的输出部分添加了Dropout()
    """
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(DropoutNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.Dropout(0.5), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.Dropout(0.3), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
        """
        这里的Sequential()函数的功能是将网络的层组合到一起。
        """

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

class BatchNet(nn.Module):
    """
    在上面的ActivationNet的基础上，增加了一个加快收敛速度的方法——批标准化
    """
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(BatchNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.BatchNorm1d(n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.BatchNorm1d(n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [24]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 定义超参数
batch_size = 64
learning_rate = 0.01
num_epoches = 50

# 数据预处理。transforms.ToTensor()将图片转换成PyTorch中处理的对象Tensor,并且进行标准化（数据在0~1之间）
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])

# 数据集的下载器
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=data_tf, download=False)
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 选择模型
#model = SimpleNet(28 * 28, 1200, 400, 10)
#model = ActivationNet(28 * 28, 300, 100, 10)
# model = BatchNet(28 * 28, 300, 100, 10)
model = DropoutNet(28 * 28, 1200, 400, 10)

print(model)
if torch.cuda.is_available():
    model = model.cuda()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 训练模型
epoch = 0
for data in train_loader:
    img, label = data
    img = img.view(img.size(0), -1)
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()
    else:
        img = Variable(img)
        label = Variable(label)
    out = model(img)
    loss = criterion(out, label)
    print_loss = loss.data.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch += 1
    if epoch % 50 == 0:
        print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))

# 模型评估
model.eval()
eval_loss = 0
eval_acc = 0
for data in test_loader:
    img, label = data
    img = img.view(img.size(0), -1)
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()

    out = model(img)
    loss = criterion(out, label)
    eval_loss += loss.data.item() * label.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.item()
print('Test Loss: {:.4f}, Acc: {:.4f}'.format(eval_loss / (len(test_dataset)),
                                              eval_acc / (len(test_dataset))))

DropoutNet(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=1200, bias=True)
    (1): Dropout(p=0.5)
    (2): ReLU(inplace)
  )
  (layer2): Sequential(
    (0): Linear(in_features=1200, out_features=400, bias=True)
    (1): Dropout(p=0.3)
    (2): ReLU(inplace)
  )
  (layer3): Sequential(
    (0): Linear(in_features=400, out_features=10, bias=True)
  )
)
epoch: 50, loss: 2.186
epoch: 100, loss: 2.005
epoch: 150, loss: 1.706
epoch: 200, loss: 1.387
epoch: 250, loss: 1.091
epoch: 300, loss: 1.067
epoch: 350, loss: 1.008
epoch: 400, loss: 0.7971
epoch: 450, loss: 0.7105
epoch: 500, loss: 0.5625
epoch: 550, loss: 0.6908
epoch: 600, loss: 0.668
epoch: 650, loss: 0.6767
epoch: 700, loss: 0.3142
epoch: 750, loss: 0.4504
epoch: 800, loss: 0.7286
epoch: 850, loss: 0.4033
epoch: 900, loss: 0.5221
Test Loss: 0.3929, Acc: 0.8920
